<a href="https://colab.research.google.com/github/Praveen76/ANOVA-Test-COVID-19/blob/master/QLoRA_LlAMA_finetuned_for_E_commerceData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers datasets peft accelerate bitsandbytes trl safetensors ipywidgets huggingface_hub python-dotenv --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
!nvidia-smi

Sat Nov  4 18:40:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:

from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

from trl import SFTTrainer

In [9]:
# Login to the Hugging Face Hub
import os
from huggingface_hub import login
from huggingface_hub import notebook_login
token="Your huggingface Token"
login(token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [12]:
import pandas as pd
import numpy as np


df=pd.read_csv('./data.csv',encoding='latin1')
df.head(2)

,ProdDescr,Q1,A1
0,High Compatibility : Compatible With iPhone 12...,What devices is this compatible with?,"It's compatible With iPhone 12, 11 , X/XsMax/X..."
1,? Fast Charger& Data Sync?-With built-in safet...,How much data can this charger transfer per se...,480 mb/s or in other words transferring 28800M...


In [21]:
data= df[['ProdDescr','Q1','A1']]
data=data.reset_index(drop=True)
data.head(2)


,ProdDescr,Q1,A1
0,High Compatibility : Compatible With iPhone 12...,What devices is this compatible with?,"It's compatible With iPhone 12, 11 , X/XsMax/X..."
1,? Fast Charger& Data Sync?-With built-in safet...,How much data can this charger transfer per se...,480 mb/s or in other words transferring 28800M...


In [22]:
data.shape

(2098, 3)

In [23]:
data.A1.iloc[90:105]

90     Based on Wikipedia's article on the RM-VL630 r...
91     It can be connected with any other device that...
92                                          Gold-Plated.
93     This USB cable is used with the Safran Morpho ...
94     According to the manufacturer the adapter uses...
95     This iPhone USB Type C Fast charge cable, made...
96     the USB 2.0 cable can deliver up to 60mps, whi...
97              50000mAh 230.95Wh lithium polymer cells.
98     The Moto E5 Play has a display size of 15.12 c...
99                                                   NaN
100                                                  NaN
101                                                  NaN
102                                                  NaN
103                                                  NaN
104                                                  NaN
Name: A1, dtype: object

In [25]:
# Convert the DataFrame to a datasets arrow dataset
from datasets import Dataset

train_data = Dataset.from_pandas(data.iloc[0:99])
train_data

Dataset({
    features: ['ProdDescr', 'Q1', 'A1'],
    num_rows: 99
})

In [48]:
test_data = Dataset.from_pandas(data.iloc[99:])
test_data

Dataset({
    features: ['ProdDescr', 'Q1', 'A1'],
    num_rows: 1999
})

In [26]:
# Set the instruction format for HuggingFaceH4/CodeAlpaca_20K
def format_instruction(sample):
	return f"""### Instruction:
You are a Personal assistant for e-commerce platforms that will give response of max 2 lines to customers' queries using Product Description:"

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
{sample['A1']}

"""

In [29]:
# The model that you want to train from the Hugging Face hub
# model_id = "HuggingFaceH4/tiny-random-LlamaForCausalLM"
model_id = "NousResearch/Llama-2-7b-hf"
# The instruction dataset to use
# dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
#dataset_name = "HuggingFaceH4/CodeAlpaca_20K"
# Dataset split
# dataset_split= "train"
# Fine-tuned model name
new_model = "QLORA-LLAMA-finetuned-for-E-commerceData"
# Huggingface repository
hf_model_repo="Praveen76/"+new_model
# Load the entire model on the GPU 0
device_map = {"": 0}



################################################################################
# bitsandbytes parameters
################################################################################
# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_double_nested_quant = False



################################################################################
# QLoRA parameters
################################################################################
# LoRA attention dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1



################################################################################
# TrainingArguments parameters
################################################################################
# Output directory where the model predictions and checkpoints will be stored
output_dir = new_model
# Number of training epochs
num_train_epochs = 15
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False
# Batch size per GPU for training
per_device_train_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1 # 2
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4 #1e-5
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule
lr_scheduler_type = "cosine" #"constant"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = False
# Save checkpoint every X updates steps
save_steps = 0
# Log every X updates steps
logging_steps = 25
# Disable tqdm
disable_tqdm= False

################################################################################
# SFTTrainer parameters
################################################################################
# Maximum sequence length to use
max_seq_length = 1024 #None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = True #False



Let's understand each parameter used above :

**Model Configuration:**
1. `model_id`: This is the identifier for the pre-trained LLAMA model you want to fine-tune.

**Bits and Bytes Parameters:**
2. `use_4bit`: Activates 4-bit precision base model loading.
3. `bnb_4bit_compute_dtype`: Specifies the compute data type for 4-bit base models (float16 in this case).
4. `bnb_4bit_quant_type`: Defines the quantization type (nf4 in this case).
5. `use_double_nested_quant`: Decides whether to activate nested quantization for 4-bit base models.

**QLoRA Parameters:**
6. `lora_r`: LoRA attention dimension.
7. `lora_alpha`: Alpha parameter for LoRA scaling.
8. `lora_dropout`: Dropout probability for LoRA layers.

**Training Arguments:**
9. `output_dir`: Directory where the fine-tuned model's predictions and checkpoints will be stored.
10. `num_train_epochs`: Number of training epochs.
11. `fp16`: Enables or disables mixed-precision training with float16.
12. `bf16`: Enables or disables mixed-precision training with bfloat16.
13. `per_device_train_batch_size`: Batch size per GPU for training.
14. `gradient_accumulation_steps`: Number of update steps to accumulate gradients for.
15. `gradient_checkpointing`: Enables or disables gradient checkpointing.
16. `max_grad_norm`: Maximum gradient norm for gradient clipping.
17. `learning_rate`: Initial learning rate for the AdamW optimizer.
18. `weight_decay`: Weight decay applied to all layers except bias/LayerNorm weights.
19. `optim`: Specifies the optimizer to use (paged_adamw_32bit in this case).
20. `lr_scheduler_type`: Type of learning rate scheduler (cosine or constant).
21. `max_steps`: Number of training steps, overriding `num_train_epochs`.
22. `warmup_ratio`: Ratio of steps for a linear warmup from 0 to learning rate.
23. `group_by_length`: Determines whether sequences are grouped into batches with the same length.
24. `save_steps`: Number of updates steps to save checkpoints.
25. `logging_steps`: Number of updates steps to log training progress.
26. `disable_tqdm`: Disables the tqdm progress bar during training.

**SFTTrainer Parameters:**
27. `max_seq_length`: Maximum sequence length to use.
28. `packing`: Determines whether multiple short examples are packed into the same input sequence to improve efficiency.

These parameters configure how the LLAMA model is fine-tuned, the precision used for training, and various training settings to control memory usage, training steps, and optimization.

You can adjust these parameters based on your specific requirements and hardware capabilities.

In [30]:
# Get the type
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_use_double_quant=use_double_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype
)

In [32]:
# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache = False, device_map=device_map)
model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [33]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
)

In [34]:
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer

# Define the training arguments
args = TrainingArguments(
    output_dir="./FineTuned-LLAMA-Model",
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size, # 6 if use_flash_attention else 4,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    optim=optim,
    logging_steps=logging_steps,
    save_strategy="epoch",
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    disable_tqdm=disable_tqdm,
    report_to="tensorboard",
    seed=42
)


In [36]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [38]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=packing,
    formatting_func=format_instruction,
    args=args,
)
# train the model
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model in local
trainer.save_model()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.932400
50,1.516300
75,1.191000
100,0.880200


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [39]:
# Merge the model and the adapters and save it
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

22244

In [ ]:
torch.cuda.empty_cache() # PyTorch thing


In [41]:
from peft import AutoPeftModelForCausalLM

new_model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# Merge LoRA and base model
merged_model = new_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.json')

In [43]:
new_modelv02 = "QLoRA-LlAMA-finetuned-for-E-commerceData"
# Huggingface repository
hf_model_repo="Praveen76/"+new_modelv02

In [74]:

# push merged model to the hub
merged_model.push_to_hub(hf_model_repo)
tokenizer.push_to_hub(hf_model_repo)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Praveen76/QLoRA-LlAMA-finetuned-for-E-commerceData/commit/97eb8134f9f8cd4ea5e58942f019c5c372e67480', commit_message='Upload tokenizer', commit_description='', oid='97eb8134f9f8cd4ea5e58942f019c5c372e67480', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
######################## Model Testing #########################

In [52]:
# Nowl let's test the model on the test data
sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the iron made of?

### Product Description:
Heavy weight iron for easy ironing. Safety Plus: Thermal Fuse|Deluxe metal cover. Super clean surface finish with pleasant aesthetics|Multiple temprature levels|Non-stick coated sole plate|Temprature setting dial|Thermostatic pilot lamp|Warranty: 2 years on product

### Response:



Generated Response:
The iron is made of stainless steel.




In [54]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
Bluetooth Selfie Stick is provided with a tiny wireless remote that can be connected to your phone and can be used up to about 15 Ft. What is the range of the Bluetooth Selfie Stick?

### Product Description:
This is a Bluetooth Enabled selfie Stick for Mobile Phone. Just click the wireless remote to capture the photo|Bluetooth Selfie Stick is provided with a tiny wireless remote that can be connected to your phone and can be used up to about 15 Ft|Ultra Light Selfie Stick can be used to Take selfies or capture your favourate videos. This multifunction tripod with selfie stick weighs only 113 Grams.. Rubber Grip : No|Tygot Mobile Selfie Stick is compact and foldable. It can be Extended up to 60 Cms long|Selfie Stand for Mobile Phone can be Rotated to 360 degree + 220 Degree Angular Adjustment

### Response:



Generated Re

In [55]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the RPM of the copper winded motor?

### Product Description:
Powerful 2400 RPM copper winded motor for quick heating and can be used vertically or horizontally (kindly refer to images)|Do not worry if you experience some burning smell when you run your room heater for the 1st time. This is due to the motor varnish getting heated for the 1st time. The problem should not recur in subsequent usage. If it does, please contact our Customer support|Caution - Always use the product with 16A socket. Using it on lower rating sockets may result in melting of socket or the plug. This is 2 KW heater. It will consume 2 units per hour when operated on high heat setting.|Air throw range of 10 feet which is ideal for small to medium sized room; Product can be placed both vertically and horizontally (kindly refer to images)|Cool, 

In [56]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What are Eveready's Zinc Carbon Batteries best suited for?

### Product Description:
Evereadys Zinc Carbon Battery are considered one of the best battery for remote controls, clocks, small toys, torches, etc.|Highly durable & reliable technology|Available in wide range of sizes - AAA, AA, D, C and 9V sizes

### Response:



Generated Response:
Zinc Carbon Batteries are best suited for remote controls, clocks, small toys, torches, etc.




In [57]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
How tall is the phone selfie stick tripod stand?

### Product Description:
64" Tall Cell Phone Tripods with 7 section adjustable telescopic poles, the phone selfie stick tripod stand is tall enough, can be adjusted from 12in to 64in. Therefore, this selfie stick for iPhone can capture a wider landscape with different Height|3 in 1 Design with Stable Base : Can be used as extendable Tripod for phone and camera as Phone Floor Stand. This Selfie Stick for iPhone can be quickly switched between the table tripod and selfie stick mode. Is compatible with gopro and Camera|360° Rotatable Selfie Stick Bluetooth : The mount of the tripod stand can freely rotate 360°, which can be quickly ratoted the screen horizontally and vertically , and the mount arm can be 190°flipped to shoot pictures at various angles.|Compact Size and Travel 

In [58]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the capacity of the KENT Amaze Electric Kettle?

### Product Description:
KENT Amaze Electric Kettle amazes with its 1.8L capacity and 1500W power;Boils water for tea, coffee, or instant soup within minutes & serve 5-6 people at a time|360° swivel technology makes pouring and serving easy;Its inner stainless steel body ensures better heat retention and durability|The plastic outer body for cool-to-touch exterior ensures easy handling;The auto shut-off features turns it off in case of overheating and dry-boiling|Size Name: Stander; Color Name: White

### Response:



Generated Response:
The KENT Amaze Electric Kettle has a capacity of 1.8L and a power of 1500W.




In [59]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What type of dirt can this vacuum cleaner perfect clean?

### Product Description:
Wet or dry, bagged or bagless - No matter it is dust, hair, daily waste, the device can clean the dirt perfectly. You don't need to change filter during operation. Please take out the cloth dust filter and use sponge filter when sucking liquid|Blower function- The blower function is suitable for drying and cleaning of narrow and hard- to-reach indoor or outdoor areas. For effective blowing results, do not use any filter in vacuum cleaner. To hold a large amount of waste, this vacuum cleaner boasts a container capacity of 10 litres|Powerful and durable  Equipped with powerful motor delivers 14KPA strong suction power and ensures long time operation. The vacuum cleaner has an impact resistant polymer tank for longer usage life|Safe buoy techn

In [61]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the maximum capacity of this rice cooker?

### Product Description:
Multi functional use- Steamed veggies and rice, various recipes you can accomplish it all with your rice cooker|Cooks up to 600 gms of raw rice or up to 5.5 cups of raw rice|Multi cooking functions-White rice, brown rice with low carb, short grain, porridge, steam, slow cook|ADVANCED FUZZY LOGIC RICE COOKER TECHNOLOGY - auto adjust temperature and timings for optimal rice cooking combined with tailored useful add on options including steam, porridge, slow cook settings Including 24 hour pre-set timer and keep warm features|Comes with 3 Litres 5-layer thick ceramic coated inner bowl with rice level lines|Smart touch button with 3D surround heating|Includes inner bowl, stainless steel steaming basket, rice measuring cup, rice spatula and soup ladle|1

In [62]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What size is the mouse pad?

### Product Description:
Large size design makes it provide plenty of room to place your mouse, keyboard and other desk items while you are playing games or working as well as protecting your desk.|The silky smooth surface help to move with high speed and pinpoint accurately. Anti-slip rubber backing offer heavy grip to prevent the mouse pad from sliding around the desk.|The mouse pad is made of natural rubber material and the durable stitched edges, which can ensure long-term use without wear, deformation and degumming. It will provide you with a comfortable experience.|Its thick comfy and foldable design, you can carry this mouse pad with you everywhere you go. Its anti fraying stitching, prevent damage from spills or rough use.|Size: 900L x 400B x 2H mm :::: Color: Black :::: Material: Rubbe

In [63]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
How long is the battery life for the JBL GO 2?

### Product Description:
5 Hours of Playtime under optimum audio settings|Wireless Bluetooth Streaming|IPX7 Waterproof design, Battery Type Lithium-ion polymer (3.7V, 730mAh), Charging time (hrs) 2.5|Built-in Noise-cancelling Speakerphone|Audio cable input|Whats in the box: 1 x JBL GO 2, 1 x Micro USB cable for charging, 1 x Safety Sheet, 1 x Quick Start Guide, 1 x Warranty Card

### Response:



Generated Response:
The JBL GO 2 has a battery life of up to 5 hours.




In [64]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the purpose of the HDMI A Male to A Male Cable?

### Product Description:
Please select appropriate display resolution & refresh rate for proper display output. HDMI A Male to A Male Cable: Supports Ethernet, 3D, 4K video and Audio Return Channel (ARC)|Connects Blu-ray players, Fire TV, Apple TV, PS4, PS3, XBox one, Xbox 360, computers and other HDMI-enabled devices to TVs, displays, A/V receivers and more|Cable allows you to share an Internet connection among multiple devices without the need for a separate Ethernet cable|Meets HDMI 2.0 standards (4K Video at 60 Hz, 2160p, 48 bit/px color depth) that supports bandwidth up to 18Gbps and backwards compatible with earlier versions|Cable Length: 6 feet (1.8 meters); 1 year limited warranty

### Response:



Generated Response:
Please use the latest version of the oper

In [65]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What type of mouse is the Lenovo 600 Bluetooth Silent Mouse?

### Product Description:
The Lenovo 600 Bluetooth Silent Mouse has Silent buttons without the annoying click sounds and has a minimalist pocket sized design you can comfortably rest your palm on. Easy to carry and fit into pockets without bulging|Blue Optical Sensor: Take your productivity to the max with a blue optical sensor that allows you to work on almost any surface, revolutionary, distraction-free silent buttons, and on-the-fly adjustable DPI controls. DPI Resolution : 3-level Adjustable (800, 1600, 2400)|Button Life: Right/Left buttons - up to 3 million click;Scroll button - up to 100,000 clicks | Number of Buttons : 4 (Left Click, Right Click, Scroll Click, DPI Switch) | Scrolling : 2-way Scroll Wheel|Multiple device connectivity & Quick pairing: Pair w

In [66]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
How much power does the immersion rod have?

### Product Description:
Power: 1500 watts|Warranty: 2 year warranty on product|Water Proof Sealed terminals for safety and longer life|Plastic handle to prevent from shock & rust|Includes: 1 Unit immersion rod, instruction manual, warranty card

### Response:



Generated Response:
The immersion rod has a power of 1500 watts.




In [67]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the maximum lifespan of Duracell batteries?

### Product Description:
Lasts up to 100% longer* (*Longevity performance compared to the Minimum Average Duration of the 2015 IEC AAA Motorized Toy test. Results may vary by device or usage pattern.)|Our best battery, suited for everyday devices with extra performance for high drain devices|Duralock technology keeps unused Duracell batteries fresh and powered for up to 10 years in ambient storage|Available in AAA, AA, C, D and 9V sizes|Superior Nylon top closure helps preventing our batteries from leakage

### Response:



Generated Response:
The maximum lifespan of Duracell batteries is 5 years.




In [68]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the maximum suction power of the cordless handheld vacuum cleaner?

### Product Description:
?High Power Suction? This hand held vacuuming cordless uses a built-in cyclone design, which makes this compact handheld vacuum cleaner can still provide 5200pa continuous strong suction; not only can it effectively clean dust, paper scraps, hair and other small garbage, but also small rubble; after many tests, it has been shown that the fully charged portable cordless vacuum cleaner can last for 20 to 25 minutes.|?Suction & Blowing Dual Function?The cordless handheld vacuum has a smart design, which not only has a enough cyclone suction and could be used on small areas like car interior, keyboard, workstation area, kitchen cabinets, but also could be used as a dust blower. One end of the machine is connected to the suction

In [72]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the maximum output of the pTron Volta USB charger?

### Product Description:
Made in India Fast Charging Wall Adaptor: This USB travel wall charger port charges at a powerful 12Watt.|Compact & Lightweight: pTron Volta dual port 12W USB charger allows charging at home or in the office via USB cable. A compatible USB cable should be used with this charger to experience fast charging|Dual USB Ports: Volta 2.4A charger comes with 2 USB slots which means that you can charge 2 devices simultaneously with no hassles. So convenient and practical application.|Multi-layer Chipset Protection - The charger comes with multi-layer chipset protection that gives protection against short circuits, over-voltage, and fire-resistance ABS body|Wide Compatibility: Input AC 100-240V, Output DC 5V/2.4A max. You can easily charge your smar

In [73]:

sample = test_data[randrange(len(test_data))]

prompt = f"""### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
{sample['Q1']}

### Product Description:
{sample['ProdDescr']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)


print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")


Prompt:
### Instruction:
Use the query below and the Product Description given to write the Response of max 2 lines to customers of e-commerce platforms:

### Query:
What is the coffee filter made of?

### Product Description:
Strength: The Coffee Filter is strong durable, sufficient in weight and thickness. It is resistant to Rust, Corrosion and Non-absorbent.|Quality: Food Grade Quality Material that does not allow the migration of hazardous substances or impart colors, odors, stains or tastes to food.|Physical Characteristics: Finished to have a smooth, zero sharp edge surface for Easy to use and clean in a dishwasher. It is an Inexpensive, portable, reusable and environmentally friendly Phin filter.|Taste is a personal subject but for Vibrant and Rich Aromatic cup of coffee we recommend to use Grounded Coffee, Coffee Granules, Coffee Beans or Filter Coffee Powder.|Also known as: Kumbakonam Coffee Filter, Mylapore Coffee Filter, Mysore Coffee Filter, Degree Coffee Maker.

### Respon